# Simulation Population

Implémentons avec les sommmes aggrégées de [2014](http://www2.impots.gouv.fr/documentation/statistiques/2042_nat/2015/revenus_2014_6e_ano.pdf)

In [14]:
import random
import numpy as np
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### Génération Table Population

In [15]:
openfisca_entry_variables = ['age','sexe','activite','1AJ', '1AS', '1AP', '1AO'] # à compléter au fur et à mesure
sample_size = 10000
index = np.arange(sample_size)
population = pd.DataFrame(columns = openfisca_entry_variables, index = index)

population.shape

(10000, 7)

### Sexe et Age

Issu de données INSEE 2016

À noter : âge = 100 correspond à 100 et plus

In [16]:
#path à revoir
reference_sexe = pd.read_csv("data/demographie/pop_age_sexe_2016.csv")
reference_sexe.rename(columns={'age_revolu': 'age'}, inplace=True)

assert all([dtype == np.int64 for dtype in reference_sexe.dtypes])

# assert all([col == population.columns for col in reference_sexe.columns]) peut-être un jour

print(reference_sexe.head())
effectif_population = sum(reference_sexe["total"])
print "Effectif de la population l'année concernée : ", effectif_population
                        

   age   homme   femme   total
0    0  391371  374179  765550
1    1  403204  385442  788646
2    2  405502  386831  792333
3    3  412383  391853  804236
4    4  416626  399632  816258
Effectif de la population l'année concernée :  66627602


In [17]:
nbr_femme = int(sum(reference_sexe['femme']/effectif_population)*sample_size)
nbr_homme = sample_size - nbr_femme
print "Nombre nécessaire de femmes dans notre échantillon : ", nbr_femme
print "Nombre nécessaire d'hommes dans notre échantillon : ", nbr_homme

#Calcul de la proportion de femmes et d'hommes par âge
liste_variable = ["femme", "homme"]
for variable in liste_variable:
    reference_sexe["proba_{0}".format(variable)] = reference_sexe[variable]/sum(reference_sexe[variable])

np.testing.assert_almost_equal(sum(reference_sexe.proba_femme), 1), "Nous voulons une probabilité"
np.testing.assert_almost_equal(sum(reference_sexe.proba_homme), 1), "Nous voulons une probabilité"

reference_sexe.head()

Nombre nécessaire de femmes dans notre échantillon :  5153
Nombre nécessaire d'hommes dans notre échantillon :  4847


,age,homme,femme,total,proba_femme,proba_homme
0,0,391371,374179,765550,0.010897,0.012120
1,1,403204,385442,788646,0.011225,0.012486
2,2,405502,386831,792333,0.011266,0.012558
3,3,412383,391853,804236,0.011412,0.012771
4,4,416626,399632,816258,0.011639,0.012902


In [18]:

population.loc[:nbr_femme].shape
population.head()

,age,sexe,activite,1AJ,1AS,1AP,1AO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Génération sexe
population.loc[:nbr_femme-1, "sexe"] = np.zeros(nbr_femme,dtype=int)
population.loc[nbr_femme:, "sexe"] = np.ones(nbr_homme, dtype=int)

#Génération de l'âge
ages = reference_sexe['age'].nunique()
population.loc[population['sexe'] == 0, "age"] = np.random.choice(ages, nbr_femme, p=reference_sexe.proba_femme)
population.loc[population['sexe'] == 1, "age"] = np.random.choice(ages, nbr_homme, p=reference_sexe.proba_homme)
population.head()

,age,sexe,activite,1AJ,1AS,1AP,1AO
0,57,0,NaN,NaN,NaN,NaN,NaN
1,56,0,NaN,NaN,NaN,NaN,NaN
2,8,0,NaN,NaN,NaN,NaN,NaN
3,87,0,NaN,NaN,NaN,NaN,NaN
4,46,0,NaN,NaN,NaN,NaN,NaN


In [27]:
# Génération sexe
population.loc[:nbr_femme-1, "sexe"] = np.zeros(nbr_femme,dtype=int)
population.loc[nbr_femme:, "sexe"] = np.ones(nbr_homme, dtype=int)

#Génération de l'âge
def simul(table_de_reference, name):
    ages = table_de_reference[name].nunique()
    population.loc[population['sexe'] == 0, name] = np.random.choice(ages, nbr_femme, p=table_de_reference.proba_femme)
    population.loc[population['sexe'] == 1, name] = np.random.choice(ages, nbr_homme, p=table_de_reference.proba_homme)

simul(reference_sexe, 'age')
population.head()

,age,sexe,activite,1AJ,1AS,1AP,1AO
0,15,0,NaN,NaN,NaN,NaN,NaN
1,8,0,NaN,NaN,NaN,NaN,NaN
2,77,0,NaN,NaN,NaN,NaN,NaN
3,77,0,NaN,NaN,NaN,NaN,NaN
4,60,0,NaN,NaN,NaN,NaN,NaN


In [21]:
#test de la probabilité TODO
proportion_finale_femme = population[population['sexe'] == 0].age.value_counts(sort=False, normalize=True)

proportion_finale_femme = pd.DataFrame(proportion_finale_femme)
proportion_finale_femme.columns = ['proportion_finale_femme']
pour_comparaison = reference_sexe.merge(proportion_finale_femme, left_on='age', right_index=True)
pour_comparaison[['proportion_finale_femme', 'proba_femme','age']]

var_ref = 'proba_femme'
var_simul = 'proportion_finale_femme'
ratio = pour_comparaison[var_simul]/pour_comparaison[var_ref]
ratio.describe()

count    100.000000
mean       1.022791
std        0.213793
min        0.703257
25%        0.909691
50%        0.989363
75%        1.083165
max        2.441866
dtype: float64

### Activité

In [22]:
reference_activite = pd.read_csv("data/demographie/activite.csv")
reference_activite.dtypes

#chiffre issu de INSEE -> comment ne pas le mettre en dur
reference_activite[['homme','femme','total']] *= 1000
effectif_population_active = sum(reference_activite['total'])
print "Effectif de la population active", effectif_population_active
reference_activite


Effectif de la population active 28727000


,pop_active,femme,homme,total
0,15-24 ans,1228000,1475000,2703000
1,25-49 ans,8563000,9274000,17837000
2,50-64 ans,3879000,4003000,7882000
3,65 ans ou plus,126000,179000,305000


In [30]:
a = pd.melt(reference_activite, id_vars=['pop_active'], value_vars=['femme', 'homme','total'], var_name = 'sexe',value_name='activite')

a['variable'] = a['pop_active'] +  a['sexe']
print(sum(a.activite[:8]))
a.head()


28727000


,pop_active,sexe,activite,variable
0,15-24 ans,femme,1228000,15-24 ans femme
1,25-49 ans,femme,8563000,25-49 ans femme
2,50-64 ans,femme,3879000,50-64 ans femme
3,65 ans ou plus,femme,126000,65 ans ou plus femme
4,15-24 ans,homme,1475000,15-24 ans homme


In [24]:
reference_sexe.head()

,age,homme,femme,total,proba_femme,proba_homme
0,0,391371,374179,765550,0.010897,0.012120
1,1,403204,385442,788646,0.011225,0.012486
2,2,405502,386831,792333,0.011266,0.012558
3,3,412383,391853,804236,0.011412,0.012771
4,4,416626,399632,816258,0.011639,0.012902


In [25]:
a["non_activite"]= 

SyntaxError: invalid syntax (<ipython-input-25-f6b1f98f8cc5>, line 1)

In [ ]:
activite_somme = {name: sum(reference_activite[name]) for name in reference_activite.columns}
activite_somme


In [ ]:
liste_variable = ["femme", "homme"]
for variable in liste_variable:
    reference_activite["proba_{0}".format(variable)] = reference_activite[variable]/reference_activite['total']

assert sum(reference_activite.proba_femme+reference_activite.proba_homme) == 4, "Ce sont des probabilités"
# à préciser car forme proba pas explicite
reference_activite

In [ ]:
#Génération population inactive jeune
population.loc[population['age'] < 15, "activite"] = 0
population[population.age < 15].activite 
#Population inactive > 65
population.loc[(population['age']>15) & (population['age'] < 24) & (population['sexe'] == 0), 'activite'] =

In [ ]:
nbr_population_active = int(sum(reference_activite["total"]/effectif_population)*sample_size)
print "Effectif populaction active dans notre échantillon ", nbr_population_active
nbr_population_inactive = sample_size - nbr_population_active
print "Effectif populaction inactive dans notre échantillon ", nbr_population_inactive

#Calcul de la proportion de femmes et d'hommes actifs par âge
liste_variable = ["femme", "homme"]
for variable in liste_variable:
    reference_activite["proba_{0}".format(variable)] = reference_activite[variable]/sum(reference_activite[variable])

assert sum(reference_activite.proba_femme) == 1, "Nous voulons une probabilité"
assert sum(reference_activite.proba_homme) == 1, "Nous voulons une probabilité"

reference_activite.head()

In [ ]:
reference_activite.T.head()

In [ ]:
population.loc[(population['sexe'] == 0) & (population['age'] > 65), 'activite'] 

In [ ]:
population.loc[(population['sexe'] == 0 &, name] = np.random.choice(ages, nbr_femme, p=table_de_reference.proba_femme)
population.loc[population['sexe'] == 1, name] = np.random.choice(ages, nbr_homme, p=table_de_reference.proba_homme)


In [ ]:
population.loc[(population['sexe'] == 0) & (population['age'] > 15), 'activite'] = np.random.choice(nb.arange(2), nbr_femme, p=table_de_reference.proba_femme)

In [ ]:
revenus = [21820704, 503723963299] # (nbr de déclarant en case 1aj, montant total de cette case) -> 2014
revenus_moy = revenus[1] / float(revenus[0])
pourcent= REVENUS[0] / float(nbr_foyer)

In [ ]:
def generate_random_cerfa():
    cerfa = {}

    if random.random() < PERCENT_REVENUS_NOT_0 * 1.3:
        cerfa['1AJ'] = max(random.gauss(14000, 23500), 0)
    else:
        cerfa['1AJ'] = 0
    return cerfa

def gradiant(a, b):
    if a > b:
        return min((a / b - 1) * random.random(), 0.5)
    else:
        return min((b / a - 1) * random.random(), 0.5)


def find_gaussian_parameters(number_not_0, total_value, distribution_percentage_null=5):
    def simulate_population(th, mu, sigma, percentage_repr):
        total_result = 0
        number_not_null = 0
        for i in range(0, int(TOTAL_DECLARATIONS * percentage_repr)):
            result = simulate_one_gaussian(th, mu, sigma)
            total_result += result
            if result > 0:
                number_not_null += 1
        return number_not_null / percentage_repr, total_result / percentage_repr

    def simulate_one_gaussian(th, mu, sigma):
        if random.random() < th:
            return max(random.gauss(mu, sigma), 0)
        return 0

    # Between 0 and 1
    number_not_0 = float(number_not_0)
    total_value = float(total_value)

    percentage_repr = 0.001
    mu = total_value / number_not_0
    sigma = mu / 2
    mu_step = mu / 2
    sigma_step = sigma / 2
    th = (1 + distribution_percentage_null / 100.0) * number_not_0 / TOTAL_DECLARATIONS
    print repr(th)
    max_number_of_simulations = 100
    for i in range(0, max_number_of_simulations):
        sim_not_0, sim_tot_value = simulate_population(th, mu, sigma, percentage_repr)
        if sim_not_0 > number_not_0:
            mu -= mu_step * gradiant(number_not_0, sim_not_0)
            # sigma -= sigma_step * gradiant(number_not_0, sim_not_0)
        else:
            mu += mu_step * gradiant(sim_not_0, number_not_0)
            # sigma += sigma_step * gradiant(sim_not_0, number_not_0)

        if sim_tot_value > total_value:
            # mu -= mu_step * gradiant(total_value, sim_tot_value)
            sigma -= sigma_step * gradiant(total_value, sim_tot_value)
        else:
            # mu += mu_step * gradiant(sim_tot_value, total_value )
            sigma += sigma_step * gradiant(sim_tot_value, total_value)
        print 'Total target ' + str(sim_tot_value/total_value) + ' not 0 target: ' + str(sim_not_0/number_not_0) + ' mu=' +  repr(mu) + ' sigma=' + repr(sigma) + ' th=' + str(th)
        mu_step = mu_step * 0.995
        sigma_step = sigma_step * 0.995
        percentage_repr = percentage_repr * 1.01


find_gaussian_parameters(21820704, 503723963299, distribution_percentage_null=5)

### Gestion des situations familiales
Idée : obtenir le nombre moyen d'enfant = 1.7

In [ ]:
# Statistiques by familly, approximated to match the declaration d'impots
# Voir la distribution réelle mais bloquer par nbenf > 4 répartition en 3 plus calage car moyenne au-dessus de la réalité
CHILDREN_PER_FAMILY = [(1, 46), (2, 38.5), (3, 12.5), (4, 2), (5, 1)]
TOTAL_CHILDREN = float(sum(w*c for c, w in CHILDREN_PER_FAMILY))
print('TOTAL_CHILDREN = ', float(sum(w*c for c, w in CHILDREN_PER_FAMILY)))
# Familles avec enfants a charge de moins de 18 ans
FAMILLES = 9321480

In [ ]:
def weighted_choice(choices):
   total = sum(w for c, w in choices)
   r = random.uniform(0, total)
   upto = 0
   for c, w in choices:
      if upto + w >= r:
         return c
      upto += w
   assert False, "Shouldn't get here"


    # Age is random between 18 and 88
    cerfa['0DA'] = int(random.random() * 70 + 18)

    # Drawing the situation
    # situation = weighted_choice(POSSIBLE_SITUATIONS)
    # cerfa[situation] = 1

    ## We only give children to married or pacces. This is an approximation
    # enfants = 0
    # if situation == 'M' or situation == 'O':
    #     if random.random() < (FAMILLES / float(POSSIBLE_SITUATIONS[0][1] + POSSIBLE_SITUATIONS[2][1])):
    #         enfants = weighted_choice(CHILDREN_PER_FAMILY)
    #
    # if enfants > 0:
    #     cerfa['F'] = enfants

    # Distribution that has a cool shape and required properties 5500, 26500

In [ ]:
situations = [['M', 12002841], ['D', 5510809], ['O', 983754], ['C', 14934477], ['V', 3997578]]
nbr_foyer = int(sum(w for c, w in situations))
print"Nombre de déclarations :",nbr_foyer

#to do : pour l'instant références implémentées en dur

pourcent_situations = situations # connaitre la proportion pour distribuer
for x in pourcent_situations :
    x[1] = float(x[1])/nbr_foyer
pourcent_situations

openfisca_entry_variables = ['1AJ', '1AS', '1AP', '1AO'] # à compléter au fur et à mesure
index = np.arange(nbr_foyer/1000)
population = pd.DataFrame(columns=openfisca_entry_variables, index = index)
len(population)
population.head()